# 4章ニューラルネットワークの学習

ニューラルネットワークの特徴は、データから学習できる点にあります。データから学習するとは、重みパラメータの値をデータから自動で決定できるということです。

損失関数は、任意の関数を用いることができますが、一般には、2 乗和誤差や交差エントロピー誤差などが用いられます。

In [1]:
import numpy as np

In [2]:
# 2乗和誤差
def sum_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [4]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 「2」を正解とする
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

print(sum_squared_error(np.array(y), np.array(t)))

y_2 = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(sum_squared_error(np.array(y_2), np.array(t)))

0.09750000000000003
0.5975


ここでは、2 つの例を示しています。ひとつ目の例は、正解を「2」として、ニュー ラルネットワークの出力が「2」で最も高い場合です。一方、2 つ目の例では、正解は 「2」ですが、ニューラルネットワークの出力は「7」で最も高くなっています。この
実験の結果で示されるように、ひとつ目の例の損失関数のほうが小さくなっており、 教師データとの誤差が小さいことが分かります。つまり、ひとつ目の例のほうが、出 力結果が教師データにより適合していることを 2 乗和誤差は示しているのです。

In [5]:
# 交差エントロピー誤差
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

中身の実装では、np.log の 計算時に、微小な値である delta を足して計算しています。これは、np.log(0) の ような計算が発生した場合、np.log(0) はマイナスの無限大を表す-inf となり、そ うなってしまうと、それ以上計算を進めることができなくなります。その防止策とし て、微小な値を追加して、マイナス無限大を発生させないようにしています。

In [6]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

print(cross_entropy_error(np.array(y), np.array(t)))

y_2 = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

print(cross_entropy_error(np.array(y_2), np.array(t)))

0.510825457099338
2.302584092994546
